---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [6]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import datetime



# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [4]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}


In [3]:
# reads the housing file and drops everything before 2000
def loadHousing():
    df = pd.read_csv("City_Zhvi_AllHomes.csv")
    idx = df.columns.get_loc('2000-01')
    allIndices = list(range(0,6))
    allIndices.extend(range(idx, len(df.columns)))
    return df[df.columns[allIndices]]

# return the part with years/quarters
def loadGDP():
    gdp = pd.read_excel("gdplev.xls", parse_cols=[4,5,6],skiprows=8, header=None, names=['Year_Q','GDP_Billion_Current_Dollar','GDP_Billion_Chained_2009'])
    # drop anything before 2000
    gdp =  gdp[gdp['Year_Q'].str.startswith("2")]
    # difference vs prev quarter
    gdp['DIFF'] = gdp['GDP_Billion_Chained_2009'].diff()
    # difference in the next quarter
    gdp['NEXT_DIFF'] = gdp['DIFF'].shift(-1)
    # prev difference 
    gdp['PREV_DIFF'] = gdp['DIFF'].shift(1)

    
    return gdp.reset_index(drop= True)
   


### Pickle the result

In [12]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    with open('university_towns.txt', 'r') as content_file:
        lines = content_file.readlines()
       
    state = None
    data = []
    #for nextLine in lines:
    for nextLine in lines:
        l = nextLine.strip()
        # is the line a beginning of new state
        if "[edit]" in l:
            state = l.split("[edit]")[0]
        else:
            # this is the campus name
            universityTown = l.split(" (")[0]
            # add to the list 
            data.append([state, universityTown])
    content_file.close()
    return pd.DataFrame(data=data, columns=["State","RegionName"])



In [16]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    x = loadGDP()
    x['recession_start'] = x.apply(lambda x: (x['DIFF']<0) & (x['NEXT_DIFF']<0),axis=1)
    return x[x['recession_start']].iloc[0,0]



In [15]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    x = loadGDP()
    # get the start of the recession
    recession_start= get_recession_start()
    x['recession_end'] = x.apply(lambda x: (x['DIFF']>0) & (x['PREV_DIFF']>0),axis=1)
    after_recession_start = x[x['Year_Q']>recession_start]
    return after_recession_start[after_recession_start['recession_end']].iloc[0,0]


In [10]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    x = loadGDP()
    #A recession bottom is the quarter within a recession which had the lowest GDP.
    start_of_recession = get_recession_start()
    end_of_recession = get_recession_end()
    recession = x[(x['Year_Q'] >= start_of_recession) & (x['Year_Q'] <= end_of_recession)]
    return recession.loc[recession['GDP_Billion_Chained_2009'].idxmin(axis=1),'Year_Q']




In [1]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    A quarter is a specific three month period, 
    Q1 is January through March, 
    Q2 is April through June, 
    Q3 is July through September, 
    Q4 is October through December.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    if there any NaN in a 3 months period, Quarter is NaN
    '''
    # load housing
    housing = loadHousing()
    # update state
    housing['State'] = housing['State'].apply(lambda x: states[x])
    # set index
    housing = housing.set_index(["State","RegionName"])
    # drop extra columns
    df = housing.iloc[:,4:]
    # convert strings to datetime
    df.columns = pd.to_datetime(df.columns)
    # group by / resample by quarter
    df = df.resample(rule='Q', axis=1, how='mean')
    # convert the names of the columns back to string ( lowercase)
    df.columns = map(lambda x: str(x).lower(), pd.PeriodIndex(df.columns))
    return df


In [200]:
def prepHousingData():
    r_start = get_recession_start()
    r_end =   get_recession_end()
    r_bottom = get_recession_bottom()
    c = convert_housing_data_to_quarters()
    start_idx = c.columns.get_loc(r_start)
    end_idx = c.columns.get_loc(r_end)
    housing_data = c.iloc[:,start_idx-1:end_idx+1]
    housing_data.columns.values[0] = 'quarter_before_recession'
    # add the ratio of before the recession to the recession bottom
    housing_data['price_ratio'] = housing_data['quarter_before_recession'] /housing_data.loc[:,r_bottom]
    return housing_data
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    housing_data = prepHousingData()
    
    h = get_list_of_university_towns()
    h = h.set_index(['State','RegionName'])
    university_housing = pd.merge(left=housing_data, right=h, left_index=True, right_index=True, how='inner')

    index_difference = housing_data.index.difference(university_housing.index)
    non_university_housing = housing_data.loc[index_difference]

    g = ttest_ind(university_housing['price_ratio'].values,non_university_housing['price_ratio'].values, nan_policy='omit')
    return (True, g.pvalue,'university town')     
#run_ttest()



/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:27: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, 0.0027240637047531249, 'university town')